In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Lightgbm
%cd ~/
!git clone --recursive https://github.com/microsoft/LightGBM
%cd LightGBM
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j2
%cd ~/
%cd LightGBM/python-package
!sudo python setup.py install --precompile

/root
fatal: destination path 'LightGBM' already exists and is not an empty directory.
/root/LightGBM
mkdir: cannot create directory ‘build’: File exists
-- Found OpenMP_C: -fopenmp  
-- Found OpenMP_CXX: -fopenmp  
-- Found OpenMP: TRUE   
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   filesystem
--   system
-- Configuring done
-- Generating done
-- Build files have been written to: /root/LightGBM
[ 92%] Built target _lightgbm
[100%] Built target lightgbm
/root
/root/LightGBM/python-package
running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying li

In [5]:
# Mecab
%cd ~/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
! bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh

/root
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 16), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.
Installing konlpy.....
     |████████████████████████████████| 19.4MB 25kB/s 
     |████████████████████████████████| 92kB 15.0MB/s 
     |████████████████████████████████| 2.7MB 68.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2019-12-14 01:16:05--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitb

In [1]:
import lightgbm as lgb
print(lgb.__version__)

2.3.2


In [2]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
base_path = '/content/drive/My Drive/dacon_14'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

(297571, 3)

In [0]:
mecab = Mecab()
# df_space['morphs'] = df_space['spacing'].apply(lambda x: mecab.morphs(x))
df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))
# df_space['text_morphs'] = df_train['text'].apply(lambda x: mecab.morphs(x))

In [0]:
# df_fea

In [0]:
from gensim.models import Word2Vec
# w2v_size = 100
# w2v_model = Word2Vec(df_fea['nouns'].values, 
#                            size=gensim_size, 
#                            window = 4, 
#                            min_count=50, 
#                            workers=8, 
#                            iter=100, 
#                            sg=1)
# ts = datetime.now().strftime('%Y%m%dT%H%M%S')
# file_name = 'model/gensim_w2v_{}'.format(ts)
# print(file_name)
# joblib.dump(w2v_model, '{}.pkl'.format(file_name))
# w2v_model.save('{}.model'.format(file_name))
# w2v_model

ts = '20191214T055747'
file_name = 'model/gensim_{}'.format(ts)

w2v_model = joblib.load(os.path.join(base_path, '{}.pkl'.format(file_name)))
w2v_size = w2v_model.wv.vectors.shape[1]


In [7]:
w2v_model.wv.vocab.keys()

dict_keys(['행성', '팀장', '행복', '주말', '오늘', '하루', '시작', '은행', '라운지', '안녕', '고객', '금일', '납부', '금액', '원', '감사', '새해', '복', '옥포', '지점', '해', '동안', '저희', '성원', '년', '소망', '일', '가정', '건강', '기원', '사랑', '가족', '정', '설', '명절', '직원', '일동', '월', '한', '현남', '전담', '대리', '무교', '과장', '아침', '눈', '세상', '적', '순간', '출근', '걱정', '어른', '인사', '전', '제', '발령', '나', '본점', '구정', '이별', '뿐', '저', '은혜', '진심', '가슴', '깊이', '간직', '정유', '뜻', '바', '지점장', '배상', '연휴', '추억', '오', '광고', '뒤', '마음가짐', '준비', '당', '상품', '자격', '기준', '심사', '완화', '정보', '전달', '수신', '희망', '경우', '거부', '답장', '목록', '등록', '번', '발송', '조치', '신청', '방법', '터치', '문자', '상담', '시', '분', '전화', '친절', '안전', '도움', '카카오톡', '서비스', '시행', '중', '친구', '추가', '후', '시중', '기존', '한도', '사용', '기', '대출', '원리금', '균등', '방식', '입금액', '부담', '채무', '건수', '관리', '말씀', '금리', '때문', '때', '필요', '파트너', '특징', '거래', '진행', '가능', '자체', '등급', '판단', '신용자', '부채', '통합', '전환', '이자', '납입', '만기', '일시', '가상', '조회', '확인', '행', '안내', '직장', '전용', '신용', '최대', '억', '만', '최저', '기업', '리스트', '차등', '적용', '연봉', '대비', 

In [8]:
w2v_size

100

In [0]:
def mean_w2v(row):
    nouns = row['nouns']
    w2v = np.zeros(w2v_size)

    
    for n in nouns:
        if n in w2v_model.wv.vocab.keys():
            w2v = np.add(w2v, w2v_model.wv[n])
            
    return w2v if len(nouns) == 0 else np.true_divide(w2v, len(nouns))

w2v_cols = ['w2v_{}'.format(i) for i in range(w2v_size)]


df_fea[w2v_cols] = df_fea.apply(mean_w2v, axis=1, result_type='expand')

In [0]:
tfidf_size = 100
tfidf_cols = ['tfidf_{}'.format(i) for i in range(tfidf_size)]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=tfidf_size)

df_tfidf = pd.DataFrame(vectorizer.fit_transform(df_fea['nouns'].apply(lambda x: ' '.join(x))).toarray(), columns=tfidf_cols)

In [12]:
df_fea.shape, df_tfidf.shape

((297571, 104), (297571, 100))

In [0]:
# df_tfidf

In [0]:
# df_fea

In [15]:
df_merged = pd.concat([df_fea.reset_index(), df_tfidf[tfidf_cols]], axis=1)
df_merged.shape

(297571, 205)

In [0]:
# df_merged.head(10)

In [0]:
# df_merged.to_csv('data_{}.csv'.format(ts))

In [18]:
cat_cols = []
fea_cols = w2v_cols + tfidf_cols + cat_cols

fea_cols

['w2v_0',
 'w2v_1',
 'w2v_2',
 'w2v_3',
 'w2v_4',
 'w2v_5',
 'w2v_6',
 'w2v_7',
 'w2v_8',
 'w2v_9',
 'w2v_10',
 'w2v_11',
 'w2v_12',
 'w2v_13',
 'w2v_14',
 'w2v_15',
 'w2v_16',
 'w2v_17',
 'w2v_18',
 'w2v_19',
 'w2v_20',
 'w2v_21',
 'w2v_22',
 'w2v_23',
 'w2v_24',
 'w2v_25',
 'w2v_26',
 'w2v_27',
 'w2v_28',
 'w2v_29',
 'w2v_30',
 'w2v_31',
 'w2v_32',
 'w2v_33',
 'w2v_34',
 'w2v_35',
 'w2v_36',
 'w2v_37',
 'w2v_38',
 'w2v_39',
 'w2v_40',
 'w2v_41',
 'w2v_42',
 'w2v_43',
 'w2v_44',
 'w2v_45',
 'w2v_46',
 'w2v_47',
 'w2v_48',
 'w2v_49',
 'w2v_50',
 'w2v_51',
 'w2v_52',
 'w2v_53',
 'w2v_54',
 'w2v_55',
 'w2v_56',
 'w2v_57',
 'w2v_58',
 'w2v_59',
 'w2v_60',
 'w2v_61',
 'w2v_62',
 'w2v_63',
 'w2v_64',
 'w2v_65',
 'w2v_66',
 'w2v_67',
 'w2v_68',
 'w2v_69',
 'w2v_70',
 'w2v_71',
 'w2v_72',
 'w2v_73',
 'w2v_74',
 'w2v_75',
 'w2v_76',
 'w2v_77',
 'w2v_78',
 'w2v_79',
 'w2v_80',
 'w2v_81',
 'w2v_82',
 'w2v_83',
 'w2v_84',
 'w2v_85',
 'w2v_86',
 'w2v_87',
 'w2v_88',
 'w2v_89',
 'w2v_90',
 'w2v_91'

In [0]:
train_size = len(df_train)

In [20]:
from sklearn.model_selection import train_test_split
model_index, hidden_index = train_test_split(range(train_size),     
    test_size=0.1, random_state=1984)
# In[4]:
df_model = df_merged.iloc[model_index]
df_hidden = df_merged.iloc[hidden_index]
df_test = df_merged.loc[train_size:]

# In[5]:
X = df_model[fea_cols].values
y = df_model['smishing'].values
print(np.unique(y, return_counts=True))

X_hidden = df_hidden[fea_cols].values
y_hidden = df_hidden['smishing'].values
print(np.unique(y_hidden, return_counts=True))


(array([0, 1]), array([249524,  16826]))
(array([0, 1]), array([27718,  1877]))


In [21]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
print(lgb.__version__)

lgb_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
[df_model.drop(c, axis=1, inplace=True) for c in df_model.columns if 'smishing_' in c]
[df_hidden.drop(c, axis=1, inplace=True) for c in df_hidden.columns if 'smishing_' in c]
[df_test.drop(c, axis=1, inplace=True) for c in df_test.columns if 'smishing_' in c]
skf = StratifiedKFold(n_splits=10)
print(skf)
cnt = 0
train_data = lgb.Dataset(X, label=y, feature_name=fea_cols, categorical_feature=cat_cols, free_raw_data=False)

2.3.2
StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


In [0]:
initscore_filename = ''
param = {
    # 'boosting':'gbdt',
    'boosting':'dart',
    'num_leaves': 7, 
    'objective': 'binary',
    'metric':'auc',
    'num_threads': 8,
    'learning_rate': 0.03,
    'is_unbalance': True,
    'bagging_fraction':0.02,
    'bagging_freq':1,
    'feature_fraction':0.01,
    'initscore_filename':initscore_filename,
    'lambda_l1':200,
    'lambda_l2':2000,
    'device_type':'gpu',
}
num_round = 10000

for train_index, valid_index in skf.split(X, y):
    cnt = cnt + 1
    print('\n', '*' * 20, cnt, '*' * 20)
    
    # init_bst_name = 'model.txt'
    # init_model = joblib.load(init_bst_name)
    # bst = None
    # init_model = bst
    train_set = train_data.subset(train_index).construct()
    valid_set = train_data.subset(valid_index).construct()
    print('train_set', np.unique(train_set.get_label(), return_counts=True))
    print('valid_set', np.unique(valid_set.get_label(), return_counts=True))
    bst = lgb.train(param, train_set, num_round, categorical_feature=cat_cols,
                    early_stopping_rounds=200, 
                    valid_sets=[train_set, valid_set],
                    verbose_eval=200,
                    # init_model=init_model,
                   )
    impt_dict = {k:v for k, v in zip(fea_cols, bst.feature_importance())}
    del_fea_cols = []
    print(sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True)[:5])
#     for r in sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True):
#         print(r)
#         if r[1] < 1:
#             del_fea_cols.append(r[0])
#     print(del_fea_cols)
#     print(len(del_fea_cols))
#     lgb.plot_importance(bst, height=0.3, figsize=(20, 100), max_num_features=100)
    df_model['smishing_{}'.format(cnt)] = bst.predict(X)        
    df_hidden['smishing_{}'.format(cnt)] = bst.predict(X_hidden)        
    df_test['smishing_{}'.format(cnt)] = bst.predict(df_test[fea_cols].values)    


 ******************** 1 ********************
train_set (array([0., 1.], dtype=float32), array([224571,  15143]))
valid_set (array([0., 1.], dtype=float32), array([24953,  1683]))


/usr/local/lib/python3.6/dist-packages/lightgbm/callback.py:188: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[200]	training's auc: 0.99935	valid_1's auc: 0.999407
[400]	training's auc: 0.999543	valid_1's auc: 0.999579
[600]	training's auc: 0.999551	valid_1's auc: 0.999559
[800]	training's auc: 0.999442	valid_1's auc: 0.999512
[1000]	training's auc: 0.999444	valid_1's auc: 0.999539
[1200]	training's auc: 0.999529	valid_1's auc: 0.999564
[1400]	training's auc: 0.999527	valid_1's auc: 0.999551
[1600]	training's auc: 0.999503	valid_1's auc: 0.999552
[1800]	training's auc: 0.999508	valid_1's auc: 0.999561
[2000]	training's auc: 0.999525	valid_1's auc: 0.999568
[2200]	training's auc: 0.999526	valid_1's auc: 0.999564
[2400]	training's auc: 0.99953	valid_1's auc: 0.999573
[2600]	training's auc: 0.999555	valid_1's auc: 0.999576
[2800]	training's auc: 0.999557	valid_1's auc: 0.999576
[3000]	training's auc: 0.999565	valid_1's auc: 0.999577
[3200]	training's auc: 0.99955	valid_1's auc: 0.999573
[3400]	training's auc: 0.999558	valid_1's auc: 0.999576
[3600]	training's auc: 0.999563	valid_1's auc: 0.999579

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/


 ******************** 2 ********************
train_set (array([0., 1.], dtype=float32), array([224571,  15143]))
valid_set (array([0., 1.], dtype=float32), array([24953,  1683]))
[200]	training's auc: 0.999365	valid_1's auc: 0.999506
[400]	training's auc: 0.999538	valid_1's auc: 0.999626
[600]	training's auc: 0.999535	valid_1's auc: 0.9996
